In [1]:
import numpy 
numpy.random.seed(1337)

In [2]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras_contrib.layers.advanced_activations import PELU, SReLU

Using TensorFlow backend.


In [3]:
batch_size=10
num_classes=37
epochs=40

In [4]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [5]:
os.chdir("E:");
path="E:/Dataset_Final(March)/One";
classes=os.listdir(path)
x=[]#Datapoints 
y=[]#labels 
for fol in classes:
    imgfiles=os.listdir(path+u'\\'+fol);
    for img in imgfiles:
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        x.append(im)
        y.append(fol)
x=numpy.array(x)
y=numpy.array(y)

In [6]:
print(x.shape)
print(y.shape)
x=x.reshape((-1,100,100,1))

(144, 100, 100)
(144,)


In [7]:
n=x.shape[0]
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

In [8]:
test_split=round(n*2/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(96, 100, 100, 1)
(96,)
(48, 100, 100, 1)
(48,)


In [9]:
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [10]:
# inputs=Input(shape=(100,100,1))
# x=Conv2D(32, (6, 6), padding='same', strides=1)(inputs)
# x=SReLU()(x)
# x=Conv2D(32, (6, 6), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2), strides=1)(x)
# x=Dropout(0.2)(x)
# x=Conv2D(24, (3, 3), padding='same', strides=1)(x)
# x=SReLU()(x)
# x=Conv2D(24, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2), strides=1)(x)
# x=Dropout(0.2)(x)

# x=Flatten()(x)#-------------------------------------------------------------------------------0.58(ep-37)
# x=Dense(256)(x)
# x=SReLU()(x)
# x=Dense(num_classes)(x)

# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [16]:
# inputs=Input(shape=(100,100,1))

# x=Conv2D(128,(3,3), padding='same')(inputs)
# x=SReLU()(x)
# x=Conv2D(64,(3,3))(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)
# #x=Activation('softmax')(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [12]:
# inputs=Input(shape=(100,100,1))
# x=Conv2D(16, (3, 3), padding='same')(inputs)
# x=Conv2D(16, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(32, (3, 3), padding='same')(x)
# x=Conv2D(32, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(64, (3, 3), padding='same')(x)
# x=Conv2D(64, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(72, (3, 3), padding='same')(x)
# x=Conv2D(72, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(94, (3, 3), padding='same')(x)
# x=Conv2D(94, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)
# x=SReLU()(x)
# x=Flatten()(x)-----------------------------------------------0.60(ep-38)
# # x=Dense(37)(x)
# # x=SReLU()(x)

# x=Dense(num_classes)(x)
# #x=SReLU()(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [13]:
# inputs=Input(shape=(100,100,1))
# x=Conv2D(16, (3, 3), padding='same')(inputs)
# x=Conv2D(16, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(32, (3, 3), padding='same')(x)
# x=Conv2D(32, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(64, (3, 3), padding='same')(x)
# x=Conv2D(64, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(72, (3, 3), padding='same')(x)
# x=Conv2D(72, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(128, (3, 3), padding='same')(x)
# x=Conv2D(128, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.5)(x)
# x=SReLU()(x)
# x=Flatten()(x)
# # x=Dense(37)(x)
# # x=SReLU()(x)--------------------------------------------------------------0.62(ep-39)

# x=Dense(num_classes)(x)
# #x=SReLU()(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [14]:
# inputs=Input(shape=(100,100,1))

# x=Conv2D(16, (3, 3), padding='same')(inputs)
# x=Conv2D(16, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=SReLU()(x)


# x=Conv2D(32, (3, 3), padding='same')(x)
# x=Conv2D(32, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=SReLU()(x)


# x=Conv2D(64, (3, 3), padding='same')(x)
# x=Conv2D(64, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=SReLU()(x)


# x=Conv2D(128, (3, 3), padding='same')(x)
# x=Conv2D(128, (3, 3), padding='same')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)
# x=SReLU()(x)




# x=Flatten()(x)
# # x=Dropout(0.5)(x)
# # x=SReLU()(x)


# # x=Dense(112)(x)
# # x=Dropout(0.5)(x)
# # x=SReLU()(x)#--------------------------------------------------------------0.65(ep-39)

# x=Dense(num_classes)(x)
# #x=SReLU()(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [17]:
inputs=Input(shape=(100,100,1))

x=Conv2D(16, (3, 3), padding='same')(inputs)
x=Conv2D(16, (3, 3), padding='same')(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=SReLU()(x)


x=Conv2D(32, (3, 3), padding='same')(x)
x=Conv2D(32, (3, 3), padding='same')(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=SReLU()(x)


x=Conv2D(64, (3, 3), padding='same')(x)
x=Conv2D(64, (3, 3), padding='same')(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=SReLU()(x)


x=Conv2D(128, (3, 3), padding='same')(x)
x=Conv2D(128, (3, 3), padding='same')(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Dropout(0.2)(x)
x=SReLU()(x)




x=Flatten()(x)
x=Dense(num_classes)(x)
output=Activation('softmax')(x)
model=Model([inputs], output)

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 100, 100, 16)      160       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 16)      2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
s_re_lu_7 (SReLU)            (None, 50, 50, 16)        160000    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 32)        4640      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 50, 50, 32)        9248      
__________

In [19]:
opt=keras.optimizers.RMSprop(lr=1e-3)

In [20]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [21]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')

In [22]:
hist=model.fit(x_train, y_train, 
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint]
               )

Train on 96 samples, validate on 48 samples
Epoch 1/40
96/96 [==============================] - 6s 64ms/step - loss: 3.6873 - acc: 0.0104 - val_loss: 3.7329 - val_acc: 0.0000e+00

Epoch 00001: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement- 1-0.00.hdf5
Epoch 2/40
96/96 [==============================] - 5s 53ms/step - loss: 3.6170 - acc: 0.0521 - val_loss: 3.8381 - val_acc: 0.0000e+00

Epoch 00002: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement- 2-0.00.hdf5
Epoch 3/40
96/96 [==============================] - 5s 55ms/step - loss: 3.3152 - acc: 0.0833 - val_loss: 3.2382 - val_acc: 0.1667

Epoch 00003: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement- 3-0.17.hdf5
Epoch 4/40
96/96 [==============================] - 5s 54ms/step - loss: 2.6756 - acc: 0.2917 - val_loss: 2.6266 - val_acc: 0.1667

Epoch 00004: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_

Epoch 34/40
96/96 [==============================] - 5s 54ms/step - loss: 0.6802 - acc: 0.6458 - val_loss: 1.5749 - val_acc: 0.5417

Epoch 00034: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-34-0.54.hdf5
Epoch 35/40
96/96 [==============================] - 5s 56ms/step - loss: 0.6633 - acc: 0.6146 - val_loss: 1.8877 - val_acc: 0.4583

Epoch 00035: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-35-0.46.hdf5
Epoch 36/40
96/96 [==============================] - 5s 55ms/step - loss: 0.6795 - acc: 0.6979 - val_loss: 1.3275 - val_acc: 0.5208

Epoch 00036: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-36-0.52.hdf5
Epoch 37/40
96/96 [==============================] - 5s 55ms/step - loss: 0.6812 - acc: 0.6146 - val_loss: 1.4889 - val_acc: 0.5208

Epoch 00037: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-37-0.52.hdf5
Epoch 38/40


In [23]:
score=model.evaluate(x_test, y_test)

48/48 [==============================] - 1s 15ms/step


In [24]:
score[1]

0.52083333333333337

In [25]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  2.95616179e-13   9.61348667e-10   1.94511740e-09 ...,   1.44961086e-05
    7.33914485e-09   9.89119550e-11]
 [  2.95740143e-09   1.25360908e-04   1.41914410e-04 ...,   1.42279750e-04
    2.65683157e-05   8.53008387e-05]
 [  4.19736511e-13   2.15289944e-07   6.14086446e-07 ...,   6.57757937e-08
    9.98494387e-01   3.57779157e-08]
 ..., 
 [  2.04257943e-12   3.54214421e-07   1.44528360e-06 ...,   1.28863036e-07
    9.98583436e-01   1.25766348e-07]
 [  2.82014856e-10   1.72238984e-07   1.05102053e-02 ...,   1.29010209e-06
    5.83405381e-07   1.05136326e-06]
 [  6.24000620e-08   1.63094039e-04   1.22856908e-03 ...,   1.08553744e-04
    3.57751414e-05   6.47220531e-07]]
[16 11 35  2  4  4 27 31 30 25 10 36 25 22  9 12 16 24 22 21  3 34 28 34 11
 29 22 24 26 32 33 24 10 16  9 33 16  4 16  9  9 12 26 29 28 35  3 33]


In [29]:
model.load_weights("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-15-0.65.hdf5")

In [30]:
score=model.evaluate(x_test, y_test)

48/48 [==============================] - 1s 14ms/step


In [31]:
score[1]

0.64583333333333337

In [32]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  1.78034694e-04   3.28120496e-03   1.34585018e-03 ...,   7.62491375e-02
    2.92167091e-03   3.83180275e-04]
 [  9.79870674e-05   1.70041583e-02   4.80077369e-03 ...,   9.19980742e-03
    5.39745670e-03   3.07324226e-03]
 [  2.27816781e-05   2.57435068e-03   3.70459794e-03 ...,   1.97455357e-03
    7.70529687e-01   1.33968086e-03]
 ..., 
 [  3.45447006e-05   3.42285424e-03   4.97723697e-03 ...,   2.61275726e-03
    7.45709658e-01   2.37681088e-03]
 [  3.25439818e-04   4.91389446e-03   4.69811037e-02 ...,   6.26867265e-03
    1.25496660e-03   2.43466836e-03]
 [  3.31278687e-04   9.64956265e-03   8.81878939e-03 ...,   6.32843701e-03
    3.32929287e-03   6.53021387e-04]]
[16 11 35  3  6  6 27 31 30 24  8 36 25 22  8 11 15 24 22 21  3 34 28 34 11
 29 22 24 26 32 33 24  8 16  8 30 15  6 15  8  8 11 26 29 28 35  3 30]


In [33]:
print(confusion_matrix(numpy.argmax(y_test, axis=1),y_pred))

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0

In [34]:

print (classification_report(numpy.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

          2       0.00      0.00      0.00         1
          3       0.67      1.00      0.80         2
          5       0.00      0.00      0.00         1
          6       0.67      1.00      0.80         2
          7       0.00      0.00      0.00         1
          8       0.33      1.00      0.50         2
          9       0.00      0.00      0.00         2
         10       0.00      0.00      0.00         1
         11       0.50      1.00      0.67         2
         13       0.00      0.00      0.00         2
         14       0.00      0.00      0.00         3
         15       0.33      0.50      0.40         2
         16       0.00      0.00      0.00         0
         20       0.00      0.00      0.00         1
         21       0.00      0.00      0.00         0
         22       0.33      1.00      0.50         1
         23       0.00      0.00      0.00         2
         24       0.75      1.00      0.86   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [41]:
import pandas

pandas.DataFrame(hist.history).to_csv("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/Figure/History.csv")        
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

In [63]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/Figure/Loss.pdf")
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available)
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']


In [64]:
pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/Figure/Accuracy.pdf")
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['bmh'])
plt.savefig(pp, format='pdf')
pp.close()